# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geopandas as gpd


# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,277.55,54,1,0.45,US,1707082003
1,1,jamestown,42.0970,-79.2353,279.95,42,0,5.14,US,1707082123
2,2,puerto natales,-51.7236,-72.4875,298.40,31,75,11.83,CL,1707082125
3,3,gidole,5.6500,37.3667,292.24,49,98,0.36,ET,1707082126
4,4,hermanus,-34.4187,19.2345,294.71,88,4,0.69,ZA,1707082126


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
plot = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size='Humidity',
    hover_cols=['City'],
    color='City',
    cmap='Category20',
    tiles='OSM',
    frame_width=700)

plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [46]:
temp_less_than_250 = (city_data_df['Max Temp'] <= 250)
print("Number of temperature less then 90 degrees: ", count_less_than_250)

Number of temperature less then 90 degrees:  0      False
1      False
2      False
3      False
4      False
       ...  
554    False
555    False
556    False
557    False
558    False
Name: Max Temp, Length: 559, dtype: bool


In [47]:
temp_less_than_250.sum()

27

In [43]:
wind_speed_less_than_3 = (city_data_df['Wind Speed'] <= 1.5)
print("Number of wind speed less than 3", wind_speed_less_than_3)

Number of wind speed less than 3 0       True
1      False
2      False
3       True
4       True
       ...  
554    False
555    False
556    False
557    False
558    False
Name: Wind Speed, Length: 559, dtype: bool


In [48]:
wind_speed_less_than_3.sum()

96

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [49]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

narrow_down_cities = temp_less_than_250 & wind_speed_less_than_3
narrow_df = city_data_df[narrow_down_cities]

# Drop any rows with null values
# YOUR CODE HERE

clean_df= narrow_df.dropna()

# Display sample data
# YOUR CODE HERE
clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
95,95,verkhnevilyuysk,63.4453,120.3167,236.71,100,18,0.61,RU,1707082163
102,102,tiksi,71.6872,128.8694,239.69,100,95,0.45,RU,1707082165
282,282,tosontsengel,48.7567,98.2839,245.14,100,100,0.22,MN,1707082198
304,304,beloyarskiy,63.7119,66.6722,243.50,100,88,1.43,RU,1707082212
315,315,usinsk,65.9939,57.5281,245.97,100,29,1.34,RU,1707082214
322,322,susuman,62.7833,148.1667,244.25,99,100,1.34,RU,1707082216
346,346,smidovich,48.6000,133.8167,241.26,100,66,0.65,RU,1707082224
355,355,khandyga,62.6667,135.6000,228.96,100,71,0.72,RU,1707082228
370,370,solnechnyy,50.7214,136.6319,245.94,98,23,1.36,RU,1707082235
392,392,taksimo,56.3444,114.8794,247.71,97,100,0.13,RU,1707082240


### Step 3: Create a new DataFrame called `hotel_df`.

In [50]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = clean_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ""
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
95,verkhnevilyuysk,RU,63.4453,120.3167,100,
102,tiksi,RU,71.6872,128.8694,100,
282,tosontsengel,MN,48.7567,98.2839,100,
304,beloyarskiy,RU,63.7119,66.6722,100,
315,usinsk,RU,65.9939,57.5281,100,
322,susuman,RU,62.7833,148.1667,99,
346,smidovich,RU,48.6000,133.8167,100,
355,khandyga,RU,62.6667,135.6000,100,
370,solnechnyy,RU,50.7214,136.6319,98,
392,taksimo,RU,56.3444,114.8794,97,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [54]:
# Set parameters to search for a hotel
radius = 10000# YOUR CODE HERE
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key,
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"# YOUR CODE HERE
    params["bias"] = f"proximity:{longitude},{latitude}"# YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)# YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json()# YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
verkhnevilyuysk - nearest hotel: No hotel found
tiksi - nearest hotel: Арктика
tosontsengel - nearest hotel: Алтай
beloyarskiy - nearest hotel: Сибирь
usinsk - nearest hotel: общежитие Лукойл-Коми
susuman - nearest hotel: No hotel found
smidovich - nearest hotel: Восток
khandyga - nearest hotel: Апельсин
solnechnyy - nearest hotel: Заря
taksimo - nearest hotel: No hotel found
chernyshevsk - nearest hotel: Карина
zeya - nearest hotel: Серебряный створ
priargunsk - nearest hotel: No hotel found
nizhniy kuranakh - nearest hotel: Самородок
mirny - nearest hotel: Зарница


,City,Country,Lat,Lng,Humidity,Hotel Name
95,verkhnevilyuysk,RU,63.4453,120.3167,100,No hotel found
102,tiksi,RU,71.6872,128.8694,100,Арктика
282,tosontsengel,MN,48.7567,98.2839,100,Алтай
304,beloyarskiy,RU,63.7119,66.6722,100,Сибирь
315,usinsk,RU,65.9939,57.5281,100,общежитие Лукойл-Коми
322,susuman,RU,62.7833,148.1667,99,No hotel found
346,smidovich,RU,48.6000,133.8167,100,Восток
355,khandyga,RU,62.6667,135.6000,100,Апельсин
370,solnechnyy,RU,50.7214,136.6319,98,Заря
392,taksimo,RU,56.3444,114.8794,97,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [59]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
plot_2 = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size='Humidity',
    color='City',
    cmap='Category20',
    tiles='OSM',
    frame_width=800,
    hover_cols=['Hotel Name', 'City', 'Country']
)


# Display the map
# YOUR CODE HERE
plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)